### Quantization aware training

#### Apply quantization aware training to whole pre-trained baseline model and check summary. All layers are now prefixed by "quant"

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer

# load baseline keras model which still has the activation layer in the end
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

annotated_model = tf.keras.models.clone_model(keras_baselineModel_activation_model)

# Register the custom layer with TensorFlow
#tf.keras.utils.get_custom_objects()['LinearSpecLayer'] = LinearSpecLayer
#tf.keras.utils.get_custom_objects()['GlobalLogExpPooling2D'] = GlobalLogExpPooling2D

# Apply quantization using `quantize_apply`
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

2023-08-09 14:42:39.665954: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 14:42:39.709780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 14:42:39.710356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 14:42:40.461894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TypeError: Error when deserializing class 'LinearSpecLayer' using config={'name': 'ADVANCED_SPEC1', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last', 'sample_rate': 48000, 'spec_shape': [64, 512], 'frame_step': 280, 'fmin': 150, 'fmax': 15000, 'frame_length': 512}.

Exception encountered: ('Keyword argument not understood:', 'data_format')

#### Does not work because of custom layer. Try to just quantize certain layers such as Conv2D for istance:

In [15]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer

# load baseline keras model which still has the activation layer in the end
keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

def apply_quantization_to_conv(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    keras_baselineModel_activation_model,
    clone_function=apply_quantization_to_conv,
)

# Apply quantization using `quantize_apply`
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

#quant_aware_model.summary()

ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`. [Error when deserializing class 'LinearSpecLayer' using config={'name': 'ADVANCED_SPEC1', 'trainable': True, 'dtype': 'float32', 'data_format': 'channels_last', 'sample_rate': 48000, 'spec_shape': [64, 512], 'frame_step': 280, 'fmin': 150, 'fmax': 15000, 'frame_length': 512}.

Exception encountered: ('Keyword argument not understood:', 'data_format')].

#### As it is during Clone, also this filtered Quantization is not possible. We add custom code for the layers:

In [14]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer, GlobalLogExpPooling2D

keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

### just custom layers
# def apply_quantization_to_custom_layers(layer):
#     if type(layer).__name__ == 'LinearSpecLayer':
#         return quantize_annotate_layer(layer)
#     elif type(layer).__name__ == 'GlobalLogExpPooling2D':
#         return quantize_annotate_layer(layer)
#     return layer

# annotated_model = tf.keras.models.clone_model(
#     keras_baselineModel_activation_model,
#     clone_function=apply_quantization_to_custom_layers,
# )
###


### just conv2D layers
def apply_quantization_to_conv(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    keras_baselineModel_activation_model,
    clone_function=apply_quantization_to_conv,
)
###

### all Layers
# def annotate_all_layers(layer):
#     return quantize_annotate_layer(layer)

# annotated_model_debug = tf.keras.models.clone_model(
#     keras_baselineModel_activation_model,
#     clone_function=annotate_all_layers,
# )
###


with quantize_scope(
    {'LinearSpecLayer': LinearSpecLayer,
    'GlobalLogExpPooling2D': GlobalLogExpPooling2D
    }):
    quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model_debug)

final spec shape:Tensor("quantize_annotate_1312/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)


RuntimeError: Layer ADVANCED_SPEC1:<class 'model.LinearSpecLayer'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

##### No matter if we just use the custom layers, all layers, or only Conv layers and additionally given own layer code, own tfmot.quantization.keras.QuantizeConfig seems to be necessary.

In [7]:
for layer in annotated_model.layers:
    print(f"Layer: {type(layer).__name__}, Class: {layer.__class__}")

Layer: InputLayer, Class: <class 'keras.src.engine.input_layer.InputLayer'>
Layer: QuantizeAnnotate, Class: <class 'tensorflow_model_optimization.python.core.quantization.keras.quantize_annotate.QuantizeAnnotate'>
Layer: BatchNormalization, Class: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
Layer: Conv2D, Class: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
Layer: BatchNormalization, Class: <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>
Layer: Activation, Class: <class 'keras.src.layers.core.activation.Activation'>
Layer: MaxPooling2D, Class: <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
Layer: AveragePooling2D, Class: <class 'keras.src.layers.pooling.average_pooling2d.AveragePooling2D'>
Layer: Concatenate, Class: <class 'keras.src.layers.merging.concatenate.Concatenate'>
Layer: Activation, Class: <class 'keras.src.layers.core.activation.Activation'>
Layer: Conv2D, Class: <class 'keras.src.lay

## Add custom layer as custom objects

In [12]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from model import LinearSpecLayer, GlobalLogExpPooling2D

keras_baselineModel_activation_path = "/home/jovyan/models/checkpoints_/baseline_two_class_model_softmax_activation/"
keras_baselineModel_activation_model = keras.models.load_model(keras_baselineModel_activation_path)

quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model


### just conv2D layers
def apply_quantization_to_conv(layer):
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

annotated_model = tf.keras.models.clone_model(
    keras_baselineModel_activation_model,
    clone_function=apply_quantization_to_conv,
)

custom_objects = {
    'LinearSpecLayer': LinearSpecLayer,
    'GlobalLogExpPooling2D': GlobalLogExpPooling2D
}
keras.utils.get_custom_objects().update(custom_objects)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

print("Done")

quant_aware_model.summary()

final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
Done
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 INPUT (InputLayer)          [(None, 144000)]             0         []                            
                                                                                                  


#### all Conv Layer have now a QuantizeWrapperV2

## Train the quant aware Model

### Load Training data and balance

In [3]:
import data as data
import numpy as np

train_data_path = "/home/jovyan/cut-data/training/"

# Load training data
print('Loading training data...', flush=True)
x_train, y_train, labels, file_paths_train = data.loadData(train_data_path)
print('...Done. Loaded {} training samples and {} labels.'.format(x_train.shape[0], y_train.shape[1]), flush=True)

# balance the training data:
print('Balance the training data...')

y_train_indices = np.argmax(y_train, axis=1)

# minimum of one class
min_samples = min(np.bincount(y_train_indices))

# reduce entries until minimum after shuffle 
balanced_x_train = []
balanced_y_train = []
balanced_file_paths_train = []
for label in np.unique(y_train_indices):
    indices = np.where(y_train_indices == label)[0]
    np.random.shuffle(indices)  # Random order for random removal of samples
    indices = indices[:min_samples]
    balanced_x_train.append(x_train[indices])
    balanced_y_train.append(y_train[indices])
    balanced_file_paths_train.extend(file_paths_train[indices])

# Combine the balanced data for all classes
balanced_x_train = np.concatenate(balanced_x_train, axis=0)
balanced_y_train = np.concatenate(balanced_y_train, axis=0)
balanced_file_paths_train = np.array(balanced_file_paths_train)

print('Balanced training data:')
print('x_train shape:', balanced_x_train.shape)
print('y_train shape:', balanced_y_train.shape)
print('file_paths_train shape:', balanced_file_paths_train.shape)

print('...Done. Loaded {} training samples and {} labels.'.format(balanced_x_train.shape[0], balanced_y_train.shape[0]), flush=True)

Loading training data...
Processed 0 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0001_127764.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0003_82178411.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0006_718263.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0010_540556.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0016_324707.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0022_355153.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0031_270434051.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/training/non_target/0.920_0043_647427.wav
Processed 800 files. Currently processing file: /home/jovyan/cut-data/traini

### Load Validation data

In [4]:
val_data_path = "/home/jovyan/cut-data/validation/"

print('Loading validation data...', flush=True)
x_val, y_val, labels, file_paths_val = data.loadData(val_data_path)
print('...Done. Loaded {} validation samples and {} labels.'.format(x_val.shape[0], y_val.shape[1]), flush=True)

Loading validation data...
Processed 0 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.920_0001_270097.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.930_0002_182583971.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.940_0004_534761.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.950_0018_226391901.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/0.970_0017_647758.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0001_0H2uMhzSitY_520.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_--ivFZu-hlc_30.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/validation/non_target/1.000_0002_2RpOd9MJjyQ_10.wav
Processed 800 files. Currently p

## Training

In [13]:
import model

%reload_ext autoreload
%autoreload 2

train_epochs = 1
train_batch_size = 1
train_learning_rate = 0.0001
    
# Number of last layers to be trained, rest will be frozen, 0 = all will be trained
train_layers_num = 0

on_epoch_end = None

print("Shape of training labels: ", balanced_x_train.shape)
print("Shape of validation labels: ", x_val.shape)

# Train model
print('Training model...', flush=True)

# Disable Model Debug
tf.debugging.disable_check_numerics()

trained_quant_aware, history = model.trainNewModel(
                                      quant_aware_model,
                                      train_layers_num,
                                      balanced_x_train, 
                                      balanced_y_train,
                                      x_val,
                                      y_val,
                                      balanced_file_paths_train,
                                      file_paths_val,
                                      epochs=train_epochs,
                                      batch_size=train_batch_size,
                                      learning_rate=train_learning_rate,
                                      on_epoch_end=on_epoch_end)

print('Done', flush=True)

Shape of training labels:  (7612,)
Shape of validation labels:  (1381,)
Training model...
Length of trainable Weightslen  55
final spec shape:Tensor("model_1/ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, None, 1), dtype=float32)
final spec shape:Tensor("model_1/ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, None, 1), dtype=float32)
7612/7612 [==============================] - 913s 119ms/step - loss: 0.0233 - accuracy: 0.9890 - prec: 0.9890 - recall: 0.9890 - val_loss: 6.0866e-05 - val_accuracy: 1.0000 - val_prec: 1.0000 - val_recall: 1.0000
Done


## Quantization

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(trained_quant_aware)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

print('Starting conversion using Quantization...', flush=True)
quant_aware_INT8_tflite = converter.convert()
print('...Done.', flush=True)

Starting conversion using Quantization...
final spec shape:Tensor("model_1/ADVANCED_SPEC1/ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
final spec shape:Tensor("ExpandDims:0", shape=(None, 128, 513, 1), dtype=float32)
INFO:tensorflow:Assets written to: /tmp/tmp3qevlpe8/assets


INFO:tensorflow:Assets written to: /tmp/tmp3qevlpe8/assets
/opt/conda/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-09-19 20:38:36.319911: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-19 20:38:36.319982: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-19 20:38:36.320517: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp3qevlpe8
2023-09-19 20:38:36.369277: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-09-19 20:38:36.369334: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp3qevlpe8
2023-09-19 20:38:36.486242: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled

...Done.


### Evaluate Size

In [15]:
interpreter = tf.lite.Interpreter(model_content=quant_aware_INT8_tflite)
interpreter.allocate_tensors()

tflite_size = len(quant_aware_INT8_tflite) / (1024 * 1024)
print(f"Size of the TFLite model: {tflite_size:.2f} MB")

Size of the TFLite model: 15.70 MB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Save Quant. Aware Model

In [16]:
# Save the quantized model:
quant_aware_INT8_tflite_path = "/home/jovyan/models/checkpoints_/quantization_aware_INT8.tflite"
with open(quant_aware_INT8_tflite_path, "wb") as f:
    f.write(quant_aware_INT8_tflite)
print("Saved!")

Saved!


### Evaluate Quant. Aware Model

In [17]:
tf.lite.experimental.Analyzer.analyze(model_path=quant_aware_INT8_tflite_path,
                                      model_content=None,
                                      gpu_compatibility=False)

=== /home/jovyan/models/checkpoints_/quantization_aware_INT8.tflite ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the REDUCE_MIN op takes
tensor #0 and tensor #140 as input and produces tensor #186 as output.

Subgraph#0 main(T#0) -> [T#633]
  Op#0 REDUCE_MIN(T#0, T#140[1]) -> [T#186]
  Op#1 SUB(T#0, T#186) -> [T#187]
  Op#2 REDUCE_MAX(T#187, T#140[1]) -> [T#188]
  Op#3 ADD(T#188, T#14) -> [T#189]
  Op#4 DIV(T#187, T#189) -> [T#190]
  Op#5 SHAPE(T#190) -> [T#191]
  Op#6 SPLIT_V(T#191, T#138[1, 1, 0], T#120[0]) -> [T#192, T#193, T#194]
  Op#7 RESHAPE(T#193, T#121[]) -> [T#195]
  Op#8 FLOOR_DIV(T#195, T#122[8]) -> [T#196]
  Op#9 PACK(T#196, T#122[8]) -> [T#197]
  Op#10 MUL(T#196, T#122[8]) -> [T#198]
  Op#11 RESHAPE(T#198, T#124[1]) -> [T#199]
  Op#12 CONCATENATION(T#192, T#199) -> [T#200]
  Op#13 CONCATENATION(T#192, T#197) -> [T#201]
  Op#14 STRIDED_SLICE(T#190, T#125[0, 0], T#200, T#137[1, 1

In [18]:
%load_ext autoreload
%autoreload 2

import evaluateTFliteModel

test_data_path = "/home/jovyan/cut-data/testing/"
batch_size = 12

evaluateTFliteModel.evaluate_tflite_model(quant_aware_INT8_tflite_path, test_data_path, batch_size)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processed 0 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.920_0001_703520.wav
Processed 100 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.920_0057_645986.wav
Processed 200 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.930_0263_741788.wav
Processed 300 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.950_0002_645965.wav
Processed 400 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/0.960_0024_103739801.wav
Processed 500 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0001_-X5Ay0Wuew0_20.wav
Processed 600 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0001_4TQzd0lB8IQ_30.wav
Processed 700 files. Currently processing file: /home/jovyan/cut-data/testing/non_target/1.000_0002_1MF9_29YUZU_10.wav